# Chapter 7: Ensemble Learning and Random Forests

Bab ini membahas **Ensemble Learning**, sebuah teknik di mana prediksi dari sekelompok prediktor (seperti *classifier* atau *regressor*) digabungkan untuk mendapatkan prediksi yang lebih baik daripada prediktor individual terbaik sekalipun. Sekelompok prediktor ini disebut *ensemble*.

Sebagai contoh, sebuah *ensemble* dari Decision Trees disebut **Random Forest**. Meskipun sederhana, ini adalah salah satu algoritma Machine Learning yang paling kuat yang tersedia saat ini.

## Voting Classifiers

Cara sederhana untuk membuat *classifier* yang lebih baik adalah dengan menggabungkan prediksi dari beberapa *classifier* dan memprediksi kelas yang mendapatkan suara terbanyak. Ini disebut **voting classifier**.

* **Hard Voting:** Memprediksi kelas yang paling banyak dipilih oleh *classifier* individual.
* **Soft Voting:** Memprediksi kelas dengan probabilitas kelas rata-rata tertinggi. Ini seringkali berkinerja lebih baik karena memberikan bobot lebih pada suara yang sangat yakin.

In [ ]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=500, noise=0.30, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

log_clf = LogisticRegression(solver="lbfgs", random_state=42)
rnd_clf = RandomForestClassifier(n_estimators=100, random_state=42)
svm_clf = SVC(gamma="scale", random_state=42, probability=True)

voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting='soft'
)
voting_clf.fit(X_train, y_train)

## Bagging and Pasting

Pendekatan lain adalah menggunakan algoritma pelatihan yang sama untuk setiap prediktor tetapi melatihnya pada subset acak yang berbeda dari set pelatihan.

* **Bagging (Bootstrap Aggregating):** Pengambilan sampel dilakukan **dengan** penggantian (*with replacement*).
* **Pasting:** Pengambilan sampel dilakukan **tanpa** penggantian (*without replacement*).

Setelah semua prediktor dilatih, *ensemble* dapat membuat prediksi dengan menggabungkan prediksi dari semua prediktor (biasanya menggunakan *mode* statistik untuk klasifikasi atau rata-rata untuk regresi).

### Out-of-Bag (oob) Evaluation
Dengan *bagging*, beberapa *instance* mungkin tidak pernah terpilih untuk pelatihan pada prediktor tertentu. *Instance* ini disebut *out-of-bag (oob)*. Kita dapat menggunakan *instance oob* ini untuk mengevaluasi kinerja *ensemble* tanpa memerlukan *validation set* terpisah.

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    max_samples=100, bootstrap=True, n_jobs=-1, oob_score=True)

bag_clf.fit(X_train, y_train)
print(bag_clf.oob_score_)

## Random Forests

**Random Forest** adalah sebuah *ensemble* dari Decision Trees, umumnya dilatih melalui metode *bagging*.

Random Forest memperkenalkan keacakan ekstra saat menumbuhkan pohon: alih-alih mencari fitur terbaik saat membagi sebuah *node*, ia mencari fitur terbaik di antara subset acak dari fitur-fitur. Ini menghasilkan keragaman pohon yang lebih besar, yang menukar *bias* yang lebih tinggi dengan *variance* yang lebih rendah, menghasilkan model yang secara keseluruhan lebih baik.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1)
rnd_clf.fit(X_train, y_train)

## Boosting

**Boosting** mengacu pada metode *ensemble* apa pun yang dapat menggabungkan beberapa *weak learners* menjadi satu *strong learner*. Ide umumnya adalah melatih prediktor secara berurutan, masing-masing mencoba memperbaiki pendahulunya.

### AdaBoost (Adaptive Boosting)
Prediktor baru memberi perhatian lebih pada *instance* pelatihan yang salah diklasifikasikan oleh prediktor sebelumnya. Ini dicapai dengan meningkatkan bobot relatif dari *instance* yang salah diklasifikasikan.

### Gradient Boosting
Metode ini mencoba memasang prediktor baru pada *residual errors* (kesalahan sisa) yang dibuat oleh prediktor sebelumnya.

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingRegressor

# Contoh AdaBoost
ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1), n_estimators=200,
    algorithm="SAMME.R", learning_rate=0.5)
ada_clf.fit(X_train, y_train)

# Contoh Gradient Boosting (untuk regresi)
gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=120, learning_rate=0.1)
# gbrt.fit(X, y) # Perlu data regresi

## Stacking

**Stacking** (*stacked generalization*) adalah metode *ensemble* yang didasarkan pada ide sederhana: alih-alih menggunakan fungsi sepele (seperti voting) untuk menggabungkan prediksi dari semua prediktor dalam *ensemble*, mengapa kita tidak melatih sebuah model untuk melakukan agregasi ini?

Prosesnya melibatkan pemisahan *training set*. *Layer* pertama dari prediktor dilatih pada satu bagian, kemudian membuat prediksi pada bagian lain. Prediksi-prediksi ini kemudian digunakan sebagai fitur masukan untuk melatih model akhir yang disebut **blender** atau **meta-learner**.